<a href="https://colab.research.google.com/github/gibsongGH/Gurobi/blob/main/ResourceAssignmentProblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gurobi tutorial:  https://www.gurobi.com/resources/ch4-linear-programming-with-python/

In [4]:
!pip install gurobipy
from gurobipy import *

### Dataset and Gurobi Model

In [5]:
# resources and jobs sets
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

In [6]:
# matching score data
combinations, ms = multidict({
    ('Carlos', 'Tester'): 53,
    ('Carlos', 'JavaDeveloper'): 27,
    ('Carlos', 'Architect'): 13,
    ('Joe', 'Tester'): 80,
    ('Joe', 'JavaDeveloper'): 47,
    ('Joe', 'Architect'): 67,
    ('Monika', 'Tester'): 53,
    ('Monika', 'JavaDeveloper'): 73,
    ('Monika', 'Architect'): 47
})

In [7]:
# Declare and initialize model
m = Model('RAP')

Restricted license - for non-production use only - expires 2026-11-23


### Decision Variables and Constraints

In [8]:
# Create decision variables for the RAP model
# "tupledict" object x contains the newly created variables
x = m.addVars(combinations, name='assign')

In [9]:
# Create jobs constraints
# (Tester = 1): x1,1 + x2,1 + x3,1 = 1
# (Java Developer = 2): x1,2 + x2,2 + x3,2 = 1
# (Architect = 3): x1,3 + x2,3 + x3,3 = 1
# for each index in list of Jobs j, sum resources that can be assigned to job j
jobs = m.addConstrs((x.sum('*',j) == 1 for j in J), name='job')

In [10]:
# Create resources constraints
# (Carlos = 1): x1,1 + x2,1 + x3,1 <= 1
# (Joe = 2): x1,2 + x2,2 + x3,2 <= 1
# (Monika = 3): x1,3 + x2,3 + x3,3 <= 1
# for each index in list of Jobs j, sum resources that can be assigned to job j
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), name='resource')

### Objective function


  Maximize the total matching score of the assignments that satisfy the job and resource constraints.


  (53x1,1 + 80x2,1 + 53x3,1) + (27x1,2 + 47x2,2 + 73x3,2) + (13x1,3 + 67x2,3 + 74x3,3)

In [11]:
# The objective is to maximize total matching score of the assignments
m.setObjective(x.prod(ms), GRB.MAXIMIZE)

In [12]:
# Save model for inspection
# The write function prints the model formulation
m.write('RAP.lp')

### Solving the RAP problem

In [13]:
# Run optimization engine
# optimize function invokes the optimize method on the model object "m"
m.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xb343b6eb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+32   1.800000e+31   4.600000e+02      0s
       5    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.930000000e+02


In [18]:
# Display optimal values of decision variables
def print_solution(model):
    for var in model.getVars():
        if abs(var.x) > 1e-6:
            print("{0}: {1}".format(var.varName, var.x))
    print('Total matching score: {0}'.format(model.objVal))
    return None

# display optimal values of decision variables
print_solution(m)

assign[Joe,Tester]: 1.0
assign[Monika,JavaDeveloper]: 1.0
assign[Ada,Architect]: 1.0
Total matching score: 253.0


### New resource

In [16]:
# Consider Ada is now available and has mastered all three roles
new_scores = {('Ada', 'Tester'):100, ('Ada', 'JavaDeveloper'):100, ('Ada', 'Architect'):100}

# Add Ada to the job constraints
for key, val in new_scores.items():
    r, j = key
    x[key] = m.addVar(obj=val,
                      name='assign[{0},{1}]'.format(r,j),
                      column=Column([1], [m.getConstrByName('job[{0}]'.format(j))]))
    m.addConstr(x.sum('Ada', '*') <= 1, name='resource[Ada]')


In [19]:
# reoptimize
m.optimize()
# display optimal values of decision variables
print_solution(m)

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 9 rows, 12 columns and 27 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.530000000e+02
assign[Joe,Tester]: 1.0
assign[Monika,JavaDeveloper]: 1.0
assign[Ada,Architect]: 1.0
Total matching score: 253.0
